In [1]:
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans

import time
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

In [2]:
def propEmbed(A, alpha = 0.95):
    A = A / A.sum(axis = 1)
    n = A.shape[0]
    AI = np.linalg.inv(np.eye(n) - A * alpha)
    X = (1 - alpha) * AI
    return X


def modularity_matrix(adj):
    w_in = adj.sum(dim=0, keepdim=True)
    w_out = adj.sum(dim=1, keepdim=True)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

def modularity(Q, partition):
    return (Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum()

In [3]:
G =  nx.karate_club_graph()
#G = nx.les_miserables_graph()
A = nx.to_numpy_array(G)
X = propEmbed(A, alpha = 0.85)
c = KMeans(n_clusters = 4, n_init=100).fit(X.transpose()).labels_
adj = torch.FloatTensor(A)
Q = modularity_matrix(adj)
print(modularity(Q, c).item())

0.4197896122932434


In [4]:
class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNNLayer, self).__init__()
        self.weight1 = nn.Parameter(torch.randn(in_features, out_features)) # 0.5 * torch.eye(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(in_features, out_features)) # -0.5 * torch.ones(1, out_features))
        self.dropout = dropout

    def forward(self, input):
        v1 = torch.mm(input, self.weight1)
        output = v1 + self.bias
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class GNN_MLP(nn.Module):
    def __init__(self, in_features, out_features, diag=False, dropout=0.0):
        super(GNN_MLP, self).__init__()
        self.gc1 = GNNLayer(in_features, out_features, dropout)

    def forward(self, x):
        x = self.gc1(x)
        x = nn.Softmax(dim=1)(x)
        #x = 1.0 + x - x.max(dim=-1, keepdim=True).values
        #x = torch.clamp(x, 0, 1)
        #x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x

In [5]:
features = torch.FloatTensor(X.transpose())
seed = 7
np.random.seed(seed)
torch.manual_seed(seed)
t_total = time.time()
n_comm = 4 #features.shape[-1]
model = GNN_MLP(features.shape[1], n_comm)
lr = 0.002
n_epochs = 30000
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(n_epochs):
    t_1run = time.time()
    optimizer.zero_grad()
    out_embed = model(features)
    C = out_embed#[:, :n_comm]
    Q1 = torch.mm(C.T, Q)
    Q2 = torch.mm(Q1, C)
    loss = torch.trace(Q2)
    loss = -loss
    loss.backward()
    optimizer.step()
    if epoch == 0 or loss < best_loss:
        best_loss = loss #- torch.trace(Q)
        best_C = C.data
        best_embed = out_embed.data
        best_epoch = epoch
    if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
        #optimizer = optim.Adam(model.parameters(), lr=lr)
        print('Epoch: {:04d}'.format(epoch + 1),
                'Modularity: {:.8f}'.format(-best_loss.item()),
                'time: {:.4f}s'.format(time.time() - t_1run))
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print(best_loss)
print(best_embed)


Epoch: 0001 Modularity: -0.01298801 time: 0.1302s
Epoch: 1501 Modularity: 0.40277466 time: 0.0005s
Epoch: 3001 Modularity: 0.41404045 time: 0.0005s
Epoch: 4501 Modularity: 0.41629767 time: 0.0005s
Epoch: 6001 Modularity: 0.41696185 time: 0.0005s
Epoch: 7501 Modularity: 0.41952392 time: 0.0005s
Epoch: 9001 Modularity: 0.41967881 time: 0.0005s
Epoch: 10501 Modularity: 0.41973844 time: 0.0005s
Epoch: 12001 Modularity: 0.41976547 time: 0.0005s
Epoch: 13501 Modularity: 0.41977817 time: 0.0006s
Epoch: 15001 Modularity: 0.41978419 time: 0.0005s
Epoch: 16501 Modularity: 0.41978702 time: 0.0005s
Epoch: 18001 Modularity: 0.41978842 time: 0.0005s
Epoch: 19501 Modularity: 0.41978908 time: 0.0005s
Epoch: 21001 Modularity: 0.41978940 time: 0.0005s
Epoch: 22501 Modularity: 0.41978949 time: 0.0005s
Epoch: 24001 Modularity: 0.41978958 time: 0.0005s
Epoch: 25501 Modularity: 0.41978961 time: 0.0006s
Epoch: 27001 Modularity: 0.41978964 time: 0.0005s
Epoch: 28501 Modularity: 0.41978967 time: 0.0005s
Epoch: